In [10]:
import base64
import io
import requests
from PIL import Image
import streamlit as st
import replicate
# Load an image (replace with your test image path)
with open("bill.png", "rb") as f:
    image_bytes = f.read()

# Encode image in base64
img_str = base64.b64encode(image_bytes).decode("utf-8")
data_url = f"data:image/png;base64,{img_str}"

API_KEY = st.secrets["REPLICATE_API_KEY"]["value"]
  # or use your secret from st.secrets if testing in Streamlit
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

model_version = "highwaywu/image-forgery-detection:ab6f81afdf0de95354d44b61c18f4dfe31dc0ad83da8b0406d57afff8f6ace08"
data = {
    "version": model_version,
    "input": {"image": data_url}
}

prediction_url = "https://api.replicate.com/v1/predictions"
response = requests.post(prediction_url, headers=headers, json=data)
print(response.status_code)
print(response.text)


422
{"title":"Invalid version or not permitted","detail":"The specified version does not exist (or perhaps you don't have permission to use it?)","status":422}



In [6]:
print(st.secrets["REPLICATE_API_KEY"])

{'value': 'r8_8nzZe40DEeSsxsPE7wRpw38xNLhfR871G8Tzm'}


In [5]:
"""
experiment_aligned_table.py

This script generates a PDF using FPDF to experiment with column widths
and multi-line wrapping for a table that mimics an invoice summary.
It uses an aligned multi-cell approach so that all cells in a row
have the same height even if the text wraps onto multiple lines.
The resulting PDF is saved in the current directory.
"""

import os
from fpdf import FPDF

def wrap_text(text, max_width, pdf):
    """
    Wrap text so that each line's width is less than max_width.
    Returns a string with newline characters inserted.
    """
    words = text.split(" ")
    lines = []
    current_line = ""
    for word in words:
        test_line = current_line + (" " if current_line else "") + word
        if pdf.get_string_width(test_line) <= max_width:
            current_line = test_line
        else:
            if current_line:
                lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)
    return "\n".join(lines)

def create_pdf_experiment():
    # Initialize PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Settings
    font_size = 10
    line_height = 5  # height for each text line
    pdf.set_font("Arial", size=font_size)

    # Define column widths and headers for the invoice summary table
    col_widths = [15, 50, 42, 25, 30, 30]  # [Bill ID, Store Name, GSTIN, Date, Category, Total Amount]
    headers = ["Bill ID", "Store Name", "GSTIN", "Date", "Category", "Total Amount"]

    # Draw table headers (single line is assumed for headers)
    for i, header in enumerate(headers):
        pdf.cell(col_widths[i], line_height + 2, header, border=1, align="C")
    pdf.ln(line_height + 2)

    # Sample invoice data for the table
    data = [
        {
            "id": "1",
            "store_name": "Uber India Systems Private Limited",
            "gstin": "29ABCDE1234F2Z5",
            "date": "01/04/2023",
            "category": "Travel",
            "total_amount": "1234.56"
        },
        {
            "id": "2",
            "store_name": "Some Very Long Store Name That Might Need Wrapping",
            "gstin": None,
            "date": "15/04/2023",
            "category": "Food",
            "total_amount": "789.10"
        },
        {
            "id": "3",
            "store_name": "Store 3",
            "gstin": "07XYZ9876Q1P2R3",
            "date": "20/04/2023",
            "category": "Office Supplies",
            "total_amount": "456.00"
        }
    ]

    # Process each row in the table
    for row in data:
        # Retrieve each field with fallback values
        row_data = [
            row.get("id") or "N/A",
            row.get("store_name") or "N/A",
            row.get("gstin") or "N/A",
            row.get("date") or "N/A",
            row.get("category") or "N/A",
            row.get("total_amount") or "0"
        ]
        
        # Wrap text in each cell and calculate maximum number of lines in this row
        wrapped_cells = []
        max_lines = 1
        for i, cell in enumerate(row_data):
            # Allow a small padding (2 units) inside each cell
            wrapped = wrap_text(cell, col_widths[i] - 2, pdf)
            wrapped_cells.append(wrapped)
            num_lines = len(wrapped.split("\n"))
            if num_lines > max_lines:
                max_lines = num_lines
        row_height = line_height * max_lines

        # Save starting position for the row
        x_start = pdf.get_x()
        y_start = pdf.get_y()

        # Draw each cell in the row
        for i, cell in enumerate(wrapped_cells):
            x_current = pdf.get_x()
            # Print the multi-line text using multi_cell
            pdf.multi_cell(col_widths[i], line_height, cell, border=0, align='L')
            # Reset position to top-left of current cell for border drawing
            pdf.set_xy(x_current, y_start)
            # Draw the cell border with calculated row height
            pdf.rect(x_current, y_start, col_widths[i], row_height)
            # Move to next cell position
            pdf.set_xy(x_current + col_widths[i], y_start)
        # Move down to the next row after completing all cells
        pdf.ln(row_height)

    # Save the PDF to the current directory
    output_pdf = "aligned_table_experiment.pdf"
    pdf.output(output_pdf)
    print("PDF generated in the current directory as:", output_pdf)

if __name__ == "__main__":
    create_pdf_experiment()


PDF generated in the current directory as: aligned_table_experiment.pdf
